In [2]:
import netCDF4

import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import imageio
from matplotlib.pyplot import figure
from tqdm import tqdm
from get_data_funcs import get_open_DAP_precipitation

%matplotlib inline

In [26]:
from matplotlib.pyplot import get
year = "2020"
month="07"
day = ""
time_start = 40
time_end = 200
total_steps = time_end-time_start

lon_s = 1300
lon_e = 1600
URL = f"https://thredds.met.no/thredds/dodsC/remotesensing/reflectivity-nordic/{year}/{month}/yrwms-nordic.mos.pcappi-0-dbz.noclass-clfilter-novpr-clcorr-block.laea-yrwms-1000.{year}{month}{day}.nc?time[{time_start}:{time_end}],lon[{lon_s}:{lon_e}][534:647],lat[{lon_s}:{lon_e}][534:647],lwe_precipitation_rate[{time_start}:{time_end}][{lon_s}:{lon_e}][534:647]"

dataset=  xr.open_dataset(URL)

prec_arr = dataset["lwe_precipitation_rate"]
max_prec = np.max(prec_arr.values)
time_arr = dataset["time"]
lat_arr = dataset["lat"].values
lon_arr = dataset["lon"].values

# oslo_lat = 200
# oslo_lon=50
# lillehammer_lat = 80
# lillehammer_lon=35

# print(lat_arr[oslo_lat,oslo_lon])
# print(lon_arr[oslo_lat,oslo_lon])

# print(lat_arr[lillehammer_lat,lillehammer_lon])
# print(lon_arr[lillehammer_lat,lillehammer_lon])


# print(lat_arr.shape)
# print(lon_arr.shape)
# print(lat_arr)
# print(lon_arr)


timesteps = np.arange(0,total_steps, 1)
filenames = []

cmap = "PuBuGn"
for step in tqdm(timesteps):
    
    precipAtTimestep = prec_arr[step,:,:]
    #standarizr for plotting:
    # Img = (precipAtTimestep - np.mean(precipAtTimestep)) / np.std(precipAtTimestep)
    figure(figsize=(10, 10), dpi=80)
    #ds.lwe_precipitation_rate.isel(time_1=step).plot(cmap="pink")  
    plt.imshow(precipAtTimestep, cmap= cmap)
    plt.clim(0,max_prec) 
    clb=plt.colorbar()
    clb.ax.set_title('mm/h')    # plt.xticks(lon_arr[:,-1])
    # plt.yticks(lat_arr[0,:])
    plt.title(f"{time_arr.data[step]}")
    plt.scatter([oslo_lon,lillehammer_lon],[oslo_lat,lillehammer_lat], marker='^')
    plt.annotate("Oslo", (oslo_lon,oslo_lat))
    plt.annotate("Lillehammer", (lillehammer_lon,lillehammer_lat))

    filename = f'{step}.png'
    filenames.append(filename)
    
    # save frame
    plt.savefig(filename)
    plt.close()
    

# build gif (https://towardsdatascience.com/basics-of-gifs-with-pythons-matplotlib-54dd544b6f30)
with imageio.get_writer(f'precp_{year}_{month}_{day}.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)
  

100%|██████████| 160/160 [00:39<00:00,  4.07it/s]


In [ ]:
  """
    59.99467
    10.882735

    61.109974
    10.646571
    """